### How to test Burla in Jupyter

- run `make test-jupyter`
- open this file and just freestyle the tests

In [3]:
import math
import heapq
import random
from time import sleep

from burla import remote_parallel_map


def _normally_distributed_random_numbers(quantity):

    def clamp(x, lower=0.1, upper=60):
        return max(lower, min(x, upper))

    def box_muller():
        u1 = random.random()
        u2 = random.random()
        z = math.sqrt(-2 * math.log(u1)) * math.cos(2 * math.pi * u2)
        return z

    mean = -30
    std_dev = 20
    samples = []

    for _ in range(quantity):
        val = clamp(mean + box_muller() * std_dev)
        samples.append(val)

    return samples


def _min_total_runtime(sleep_times, num_workers):
    workers = [0] * num_workers
    heapq.heapify(workers)
    for t in sorted(sleep_times, reverse=True):
        earliest = heapq.heappop(workers)
        heapq.heappush(workers, earliest + t)
    return max(workers)


def _max_total_runtime(sleep_times, num_workers):
    chunk_size = len(sleep_times) // num_workers
    remainder = len(sleep_times) % num_workers

    chunks = []
    start = 0
    for i in range(num_workers):
        end = start + chunk_size + (1 if i < remainder else 0)
        chunks.append(sum(sleep_times[start:end]))
        start = end

    return max(chunks)


n_workers = 2
my_inputs = _normally_distributed_random_numbers(100)
print(f"\nsum of all sleeps: {sum(my_inputs)}")
print(f"lowest possible runtime: {_min_total_runtime(my_inputs, n_workers)}")
print(f"highest possible runtime: {_max_total_runtime(my_inputs, n_workers)}")


sum of all sleeps: 48.913049312418146
lowest possible runtime: 24.470033209257934
highest possible runtime: 32.1654064212402


In [ ]:

def simple_test_function(sleep_time):
    sleep(sleep_time)
    return sleep_time

remote_parallel_map(simple_test_function, my_inputs)